<a href="https://colab.research.google.com/github/Ajlag/cifar10_neural_network/blob/main/Normalizacija.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
from google.colab import files
upload = files.upload()

Saving predvidjanjeteksta.csv to predvidjanjeteksta.csv


In [22]:
data = pd.read_csv('predvidjanjeteksta.csv')

In [33]:
# Obrada nevalidnih vrednosti
data["Output Token"].fillna("unknown", inplace=True)
data["Output Token"] = pd.to_numeric(data["Output Token"], errors="coerce")
data["Output Token"].fillna(-1, inplace=True)


In [34]:
# Prilagođavanje oznaka tako da budu u dozvoljenom opsegu
labels = data["Output Token"].values.astype(int)
labels[labels == -1] = 0  # Zamenjivanje svih oznaka koje su -1 sa 0 ili nekom drugom vrednošću


In [35]:
# Izdvajanje tekstova i oznaka
texts = data["Input Token"].values


In [36]:
# Tokenizacija teksta
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [37]:
# Podešavanje duljine sekvenci
max_sequence_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')


In [38]:
# Izgradnja modela RNN
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 128
rnn_units = 256

In [39]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_sequence_length),
    tf.keras.layers.LSTM(rnn_units),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])


In [40]:

# Kompilacija modela
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [41]:
# print(data["Output Token"].unique())
model.fit(sequences, labels, epochs=10, batch_size=64, validation_split=0.2)


Epoch 1/10
13/13 [==============================] - 4s 77ms/step - loss: 5.9712 - accuracy: 0.9228 - val_loss: 5.2977 - val_accuracy: 1.0000
Epoch 2/10
13/13 [==============================] - 0s 30ms/step - loss: 3.1022 - accuracy: 1.0000 - val_loss: 0.3400 - val_accuracy: 1.0000
Epoch 3/10
13/13 [==============================] - 0s 30ms/step - loss: 0.0337 - accuracy: 1.0000 - val_loss: 0.0150 - val_accuracy: 1.0000
Epoch 4/10
13/13 [==============================] - 0s 29ms/step - loss: 7.4063e-05 - accuracy: 1.0000 - val_loss: 0.0065 - val_accuracy: 1.0000
Epoch 5/10
13/13 [==============================] - 0s 31ms/step - loss: 2.3730e-06 - accuracy: 1.0000 - val_loss: 0.0047 - val_accuracy: 1.0000
Epoch 6/10
13/13 [==============================] - 0s 31ms/step - loss: 1.2645e-06 - accuracy: 1.0000 - val_loss: 0.0043 - val_accuracy: 1.0000
Epoch 7/10
13/13 [==============================] - 0s 29ms/step - loss: 9.5592e-07 - accuracy: 1.0000 - val_loss: 0.0042 - val_accuracy: 1.00

In [ ]:
# Evaluacija modela na testnom skupu podataka (opciono)


In [46]:
# Evaluacija modela
loss, accuracy = model.evaluate(sequences, labels)
print(f"Train Loss: {loss}, Train Accuracy: {accuracy}")

# Predviđanje normalizacije novih tekstova
new_texts = ["neki novi tekstovi ovde", "još neki primeri", "tekst koji treba normalizovati"]
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_sequences = pad_sequences(new_sequences, maxlen=max_sequence_length, padding='post')
predictions = model.predict(new_sequences)

# Dekodiranje predviđenih sekvenci reči u tekstualni format

decoded_predictions = []
for prediction in predictions:
    decoded_word = tokenizer.index_word.get(np.argmax(prediction))
    decoded_predictions.append(decoded_word)


# Ispisivanje predviđenih normalizovanih tekstova
for text, prediction in zip(new_texts, decoded_predictions):
    print(f"Originalni tekst: {text}")
    print(f"Predviđena normalizacija: {prediction}")
    print()

33/33 [==============================] - 0s 7ms/step - loss: 8.2600e-04 - accuracy: 1.0000
Train Loss: 0.0008259978494606912, Train Accuracy: 1.0
1/1 [==============================] - 0s 23ms/step
Originalni tekst: neki novi tekstovi ovde
Predviđena normalizacija: None

Originalni tekst: još neki primeri
Predviđena normalizacija: None

Originalni tekst: tekst koji treba normalizovati
Predviđena normalizacija: None

